In [31]:
import pandas as pd
import os
import numpy as np

In [12]:
df = pd.read_stata(r"..\data\raw\market_carrier_data\mktdata79q1to16q3.dta")

In [ ]:
df.info()
#ap1 = origin airport
#ap2 = destination airport
#cr1 = ReportingCarrier
#pax = no of Passengers
#avprc = Average Price

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19436121 entries, 0 to 19436120
Data columns (total 11 columns):
 #   Column  Dtype  
---  ------  -----  
 0   cr1     object 
 1   cr2     object 
 2   yr      int16  
 3   qtr     int8   
 4   cop     int8   
 5   ap1     object 
 6   ap2     object 
 7   pax     int32  
 8   nsdst   int16  
 9   avprc   float32
 10  avdst   float32
dtypes: float32(2), int16(2), int32(1), int8(2), object(4)
memory usage: 926.8+ MB


In [14]:
df.head()

,cr1,cr2,yr,qtr,cop,ap1,ap2,pax,nsdst,avprc,avdst
0,AL,TW,1979,1,1,ABE,ABQ,2,1737,133.500,1771.0
1,CO,UA,1979,1,1,ABE,ABQ,1,1737,167.000,1771.0
2,TW,UA,1979,1,1,ABE,ABQ,8,1737,209.125,1771.0
3,TW,UK,1979,1,1,ABE,ABQ,1,1737,100.000,1771.0
4,--,EA,1979,1,1,ABE,ABY,1,798,95.000,839.0


In [20]:
ORIGIN_AIRPORT = 'BOS'
DESTINATION_AIRPORT = 'LAX'
processed_data_dir = '../data/processed'
processed_filename = f'{ORIGIN_AIRPORT}_{DESTINATION_AIRPORT}_market_data.csv'
processed_filepath = os.path.join(processed_data_dir, processed_filename)
os.makedirs(processed_data_dir, exist_ok=True)

In [29]:
column_mapping = {
    'ap1': 'Origin',
    'ap2': 'Destination',
    'cr1': 'ReportingCarrier',
    'pax': 'Passengers',
    'avprc': 'MktFare'
}
df.rename(columns=column_mapping, inplace=True)

In [32]:
required_cols = ['Origin', 'Destination', 'ReportingCarrier', 'Passengers', 'MktFare']
if not all(col in df.columns for col in required_cols):
    print("Error: Not all required columns were found after renaming. Please check the column_mapping.")
    print("Available columns:", df.columns.tolist())
else:
    df_route = df[(df['Origin'] == ORIGIN_AIRPORT) & (df['Destination'] == DESTINATION_AIRPORT)].copy()

    # --- 4. Select and Clean ---
    # We select the newly renamed columns
    df_clean = df_route[required_cols].copy()
    df_clean = df_clean.rename(columns={'ReportingCarrier': 'Airline', 'MktFare': 'AvgFare'})

    # --- THIS IS THE ADDED LINE ---
    # Remove rows where the 'Airline' column is empty or missing
    df_clean['Airline'] = df_clean['Airline'].replace('--', np.nan)
    df_clean.dropna(subset=['Airline'], inplace=True)
    # --------------------------------

    df_clean['Passengers'] = pd.to_numeric(df_clean['Passengers'], errors='coerce')
    df_clean['AvgFare'] = pd.to_numeric(df_clean['AvgFare'], errors='coerce')
    df_clean.dropna(subset=['Passengers', 'AvgFare'], inplace=True)
    df_clean = df_clean[(df_clean['Passengers'] > 0) & (df_clean['AvgFare'] > 50)]
    df_clean['Passengers'] = df_clean['Passengers'].astype(int)
    print("Data cleaning complete.")

    # --- 5. Save Processed File ---
    df_clean.to_csv(processed_filepath, index=False)
    print(f"✅ Successfully saved clean data to: {processed_filepath}")
    print("Clean data head:")
    print(df_clean.head())

Data cleaning complete.
✅ Successfully saved clean data to: ../data/processed\BOS_LAX_market_data.csv
Clean data head:
      Origin Destination Airline  Passengers     AvgFare
20438    BOS         LAX      AA        3053  178.096954
20439    BOS         LAX      AA         258  171.670547
20440    BOS         LAX      AA           6  228.500000
20441    BOS         LAX      AA           2  256.000000
20442    BOS         LAX      AA           4  247.000000


In [33]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7858 entries, 20438 to 19311612
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Origin       7858 non-null   object 
 1   Destination  7858 non-null   object 
 2   Airline      7858 non-null   object 
 3   Passengers   7858 non-null   int32  
 4   AvgFare      7858 non-null   float32
dtypes: float32(1), int32(1), object(3)
memory usage: 307.0+ KB
